## קוד מבוא

In [ ]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### העלת משתנים להרצת הקוד

In [ ]:
path = os.getcwd()

parent = os.path.dirname(path)

software_data_folder_location = os.path.dirname(parent)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_data_folder_location)

## פונקציות

In [ ]:
from py_scripts.buildings_layer import buildings_layer
from py_scripts.remove_irrelevant_buildings import remove_irrelevant_buildings
from py_scripts.create_100_percent_for_every_AS import create_100_percent_for_every_AS
from py_scripts.create_information_at_taz import create_information_at_taz
from py_scripts.taz_that_split_lms_information import taz_that_split_lms_information

### פונקציות גלובליות

In [ ]:
from functions import make_point
from functions import up_load_df
from functions import up_load_gdb
from functions import up_load_shp

### העלת מידע

In [ ]:
stat=up_load_shp(r'{}\background_files\stat_11_cbs_2020_with_jtmt_fix.shp'.format(software_data_folder_location))

In [ ]:
stat_in_taz=stat.loc[stat['type_stat']=='in_taz']

In [ ]:
stat=stat.loc[stat['type_stat']!='in_taz']

In [ ]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

In [ ]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_data_folder_location)
bld=up_load_gdb(path,'BLDG_Clip')

In [ ]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_data_folder_location)
bld_poi=up_load_gdb(path,'POI_BLDG_Clip')

In [ ]:
taz_not_relevant=up_load_df(r'{}\background_files'.format(software_data_folder_location),'taz_not_relevant_for_pop_2020')

In [ ]:
taz_not_relevant=list(taz_not_relevant.Taz_num)

### שכבת בניינים

In [ ]:
bld_point=buildings_layer(bld, bld_poi)

### הורדת מבנים לא רלוונטים

In [ ]:
bld_point=remove_irrelevant_buildings(bld_point, taz, taz_not_relevant, stat)

### יצירת מאה אחוז לכל א"ס

In [ ]:
sum_bld_m3=create_100_percent_for_every_AS(bld_point)

### א"ס ללא פוליגון



In [ ]:
stat_in_taz=make_point(stat_in_taz).sjoin(taz[['Taz_num','geometry']])[['STAT','Taz_num']]

In [ ]:
stat_in_taz['precent_of_stat_data']=1

### איחוד לטבלה אחת



In [ ]:
col=['Taz_num', 'STAT',  'precent_of_stat_data']

In [ ]:
precent_of_stat_data=pd.concat([sum_bld_m3[col], stat_in_taz])

In [ ]:
precent_of_stat_data.to_excel(r'{}\Intermediates\precent_of_stat_data.xlsx'.format(software_data_folder_location),index=False)

###  יצירת מידע ברמת אזור תנועה

In [ ]:
stat=up_load_df(r'{}\Intermediates'.format(software_data_folder_location),'stat_cbs_jtmt_2020')

In [ ]:
taz_stat_conver=create_information_at_taz(stat, precent_of_stat_data)

### בשכבת האזורי תנועה יהיו רק אזורי תנועה שמפצלים את המידע של למס

In [ ]:
taz=taz_that_split_lms_information(taz_stat_conver, stat)

In [ ]:
save_taz_path=r'{}\Intermediates'.format(software_data_folder_location)

In [ ]:
taz.to_excel(r'{}\taz_with_pop_info.xlsx'.format(save_taz_path))